In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
#Directorio principal donde están las carpetas good_quality y bad_quality
data_dir = '/content/drive/MyDrive/Visión por computadora/Tareas/Actividad_Clase_11'

In [6]:
#Crear generadores de datos con una división del 20% para validación
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [7]:
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(240, 320),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(240, 320),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 1661 images belonging to 3 classes.
Found 415 images belonging to 3 classes.


In [8]:
from tensorflow.keras import layers, models

In [9]:
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                            kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
                            activation='relu',
                            input_shape=(240, 320, 3)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
                            kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
                            activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(units=hp.Int('dense_units', min_value=32, max_value=128, step=16),
                           activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Clasificación binaria

    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [10]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.0 MB/s eta 0:00:00


In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-11-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [13]:
# Definir la búsqueda aleatoria
tuner = RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5,
                     executions_per_trial=3,
                     directory='output',
                     project_name='mango_leaf_classification')

In [ ]:
#Realizar la búsqueda
tuner.search(train_generator, epochs=10, validation_data=validation_generator)

Trial 4 Complete [03h 06m 31s]
val_accuracy: 0.45783132314682007

Best val_accuracy So Far: 0.45783132314682007
Total elapsed time: 16h 18m 18s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
128               |128               |conv_1_filter
3                 |5                 |conv_1_kernel
32                |48                |conv_2_filter
5                 |3                 |conv_2_kernel
128               |96                |dense_units
0.0001            |0.0001            |learning_rate

Epoch 1/10
52/52 [==============================] - 1007s 19s/step - loss: -760.7556 - accuracy: 0.4491 - val_loss: -3451.1841 - val_accuracy: 0.4578
Epoch 2/10
52/52 [==============================] - 1024s 20s/step - loss: -21093.4238 - accuracy: 0.4582 - val_loss: -56153.7969 - val_accuracy: 0.4578
Epoch 3/10
52/52 [==============================] - 989s 19s/step - loss: -154006.2969 - accuracy: 0.4582 - val_loss: -299619.6562 - val_accuracy: 0.4578
Epoch 4/

In [ ]:
#Obtener los mejores hiperparámetros
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
#Construir y entrenar el modelo con los mejores hiperparámetros
model = build_model(best_hps)
history = model.fit(train_generator, epochs=50, validation_data=validation_generator)

In [ ]:
#Evaluar el modelo
eval_result = model.evaluate(validation_generator)
print(f'\nTest accuracy: {eval_result[1]}')